# Sprint15
## 論文読解

## (1) 物体検出の分野にはどういった手法が存在したか。

**抽出箇所**

Abstract
State-of-the-art object detection networks depend on region proposal algorithms to　hypothesize object locations.Advances like SPPnet and Fast R-CNN have reduced the running time of these detection networks, exposing region proposal computation as a bottleneck.

1 INTRODUCTION
Recent advances in object detection are driven by the success of region proposal methods (e.g., [4])and region-based convolutional neural networks (RCNNs) [5]. Although region-based CNNs were computationally expensive as originally developed in [5], their cost has been drastically reduced thanks to sharing convolutions across proposals [1], [2]. The latest incarnation, Fast R-CNN [2], achieves near real-time rates using very deep networks [3], when ignoring the time spent on region proposals. 

**物体検出の手法**

#### R-CNN(Region-based CNN)

参考文献：[Rich feature hierarchies for accurate object detection and semantic segmentation](https://www.cv-foundation.org/openaccess/content_cvpr_2014/papers/Girshick_Rich_Feature_Hierarchies_2014_CVPR_paper.pdf)

物体検出のタスクに対してCNNのアルゴリズムを上手く応用できないかという課題を解く先駆けとなった論文。

CNNの遅さと計算コストが問題で実行が非常に困難な部分を、R-CNNは、Selective Searchと呼ばれる物体候補(object proposal)アルゴリズムを使用することで解決した。

**アルゴリズム**

⑴物体らしさを見つける既存手法(Selective Search)を用いて、画像から領域候補(Region Proposals)を探す。

⑵領域候補の領域画像を 全て一定の大きさにリサイズして CNNにかけて特徴量を抽出する。

⑶抽出した特徴量を複数のSVMで学習し分類、回帰によってBounding Boxの正確な位置を推定する

入力：画像   / 出力：画像内の各物体の境界ボックス+ラベル

#### SPP-net(spatial pyramid pooling)

R-CNNの改良版、空間的なプーリング（Spatial Pyramid Pooling (SPP)）を用いた方法。
R-CNNでは入力画像は固定サイズでなければならない部分をSPP-netでは解決した。
SPP-Net は画像１枚から１回のCNNで特徴マップを作成し、領域候補の特徴をSPPによってベクトル化、全結合層へ送る。
スピードはGPU上にて24-102倍に高速化を実現した。

#### Fast R-CNN

参考文献：[Fast R-CNN](https://www.cv-foundation.org/openaccess/content_iccv_2015/papers/Girshick_Fast_R-CNN_ICCV_2015_paper.pdf)


入力：領域候補生成済みの画像情報___出力：各領域の物体分類と bounding box 座標とサイズ



## (2) Fasterとあるが、どういった仕組みで高速化したのか。

**抽出箇所**
4 EXPERIMENTS
4.1 Experiments on PASCAL VOC Using RPN yields a much faster detection system than using either SS or EB because of shared convolutional computations


- Convolutional Layer(特徴マップを抽出)とRegion Proposal Network（RPN）(物体領域を抽出)するネットワーク構成されています。（Abstract—から）

- 深層畳み込みニューラルネットワークを使用した計算とRPNの導入、畳み込み層をSPPnet,Fast R-CNNなどの最新の物体検出ネットワークと共有し、テスト時に畳み込みを共有することにより、計算提案の限界費用は小さくなる（(ex)画像ごとに10ミリ秒）(INTRODUCTIONから)


## (3) One-Stageの手法とTwo-Stageの手法はどう違うのか。

**抽出箇所**

4 EXPERIMENTS
4.1 Experiments on PASCAL VOC
One-Stage Detection vs. Two-Stage Proposal + Detection. The OverFeat paper [[9]](https://arxiv.org/pdf/1312.6229.pdf) proposes a detection method that uses regressors and classifiers on sliding windows over convolutional feature maps.
OverFeat is a one-stage, class-specific detection pipeline, and ours is a two-stage cascade consisting of class-agnostic proposals and class-specific detections. 


訳）OverFeat論文は、畳み込み特徴マップ上のスライディングウィンドウで回帰と分類を使用する検出方法を提案しています。


訳）OverFeatは1段階のクラス固有の検出パイプラインであり、私たちのものはクラスに依存しない提案とクラス固有の検出で構成される2段階のカスケードです。

#### One-Stage(ex:OverFeat,SSD,Yolo):

参考文献：
[OverFeat:Integrated Recognition, Localization and Detection
using Convolutional Networks](https://arxiv.org/pdf/1312.6229.pdf)



畳み込みネットワークを使用した認識、位置検出ネットワーク
one-stageは領域の推定と、物体の推定を一緒に行う。特徴量マップ場でウインドウをスライドさせながら、分類と回帰によって推定を行う。１つのモデルで推定することから推論速度が2-stageモデルと比較すると速い。しかし、2019年4月に出た論文[Object As Points](https://arxiv.org/pdf/1904.07850.pdf)でYOLOv3などよりも推論速度が速く、精度もよいCenterNetが提案された


#### Two-Stage(ex:Fast RCNN,Faster RCNN):
領域の推定と、物体の推定を分けて行う。
2-stageモデルは1-stageモデルと比較すると精度が高い。

## (4) RPNとは何か。

**抽出箇所**

3 FASTER R-CNN
3.1 Region Proposal Networks
A Region Proposal Network (RPN) takes an image (of any size) as input and outputs a set of rectangular object proposals, each with an objectness score.3 We model this process with a fully convolutional network At each sliding-window location, we simultaneously predict multiple region proposals, where the number of maximum possible proposals for each location is denoted as k. So the reg layer has 4k outputs encoding the coordinates of k boxes, and the cls layer outputs 2k scores that estimate probability of object or not object for each proposal


入力：画像___出力：Anchor boxesの情報（オブジェクトをアンカーボックスで切り取ったもの）

学習させること：
１.あるAnchor boxの中身が背景か物体か
2.物体だった場合、ground truthとどのくらいズレているか

「背景か物体か」については、ground truthとAnchor boxesのIOUを計算して、IOU<0.3なら「背景」、IOU>0.7なら「物体」とラベルを付けする。そのため、9（Anchor boxes/n_Anchor）×2(2値分類)=18クラス作られる。0.3<IOU<0.7の部分については学習に使わない。

featuer map上でスライディングウインドウ毎にアスペクト比が異なるアンカーボックスを当て、０か１に分類する。

⑴ある矩形（アンカーボックス）が物体なのか背景なのかを学習

⑵⑴で検出した場所に、具体的に何が写っているのかを学習

改良ポイントとしては、
従来、画像処理の手法であるSelective Searchを使用していたが、Faster R-CNN ではResion Proposal Network(RPN)と呼ばれるCNN構造を用いている。画像処理の手法をもディープラーニングを実装している点。

その座標を損失関数として、損失が小さくなるように学習する
学習したパラメータを元に物体の領域を提案する

**Anchor**
Anchorはfeature maps上の各点。

Anchor boxesは一つのAnchorに対して論文では3×3=9つ作られる。その際、各基準の長さごとにAnchor boxesの面積を揃える必要がある。画像からはみ出たAnchor boxesは無視される。

Anchor boxesは全部で18×25×9=4050個作られます。ここで様々な形のboxが提案されることにより、ground truthがどんな形であっても、ある程度似ているboxの候補をここで生成することが出来ます。
あとは一つ一つのAnchor boxとground truthを見比べて、その結果をRPNのoutputとするだけです！

**IOU**
IoU は Intersection(領域の共通部分) over Union(領域の和集合)

## (5) RoIプーリングとは何か。

**抽出箇所**

3 FASTER R-CNN
3.2 Sharing Features for RPN and Fast R-CNN
The RoI pooling layer in Fast R-CNN accepts the convolutional features and also the predicted bounding boxes as input, so a theoretically valid backpropagation solver should also involve gradients w.r.t. the box coordinates. 

訳）Fast R-CNNのRoIプーリングレイヤーは、畳み込み処理後の特徴マップと予測境界ボックスを入力として受け入れるため、理論的に有効な逆伝播にはボックス座標の勾配も含まれる必要があります。 
RoI (Region of Interest) プーリング
関心領域プーリング、又はRoIプーリングは畳み込みニューラルネットワークを使用する物体検出タスクで広く使用される操作です。その目的は、不均一なサイズの入力に対して最大プールを実行して、固定サイズの特徴マップ（たとえば7×7）を得ることです。2015年4月にRoss Girshickによって最初に提案されています。([3])

RoI Poolingは、畳み込み処理を行った後のfeature mapから、region proposalにあたる部分領域を「固定サイズのfeature map」として抽出する。

入力：領域候補生成済みの画像情報___出力：各領域の物体分類と bounding box 座標とサイズ

畳み込みした特徴量マップと予測領域を入力として受け取る
提案された領域の中でmaxpoolingを行う
出力スケールは固定されていて、全てそのサイズにスケーリングする

## (6) Anchorのサイズはどうするのが適切か。

**抽出箇所**

4 EXPERIMENTS
4.1 Experiments on PASCAL VOC

Table 8
settings anchor scales spect ratios mAP

1 scale, 1 ratio 128^2 1:1 65.8/
1 scale, 3 ratios 128^2 {2:1, 1:1, 1:2} 68.8/
3 scales, 1 ratio {128^2,256^2,512^2} 1:1 69.8/
3 scales, 3 ratios {128^2,256^2,512^2} {2:1, 1:1, 1:2} 69.9

3scales,3ratios→mAPは69.9___
３scales,1ratio→mAPは69.8

**適切なAnchorのサイズ**

3スケールの3ratioか3スケールの1ratio

## (7) 何というデータセットを使い、先行研究に比べどういった指標値が得られているか。

**抽出箇所**

4 EXPERIMENTS

**使用データセット**

PASCAL VOC 2007/
PASCAL VOC 2012/
Microsoft COCO object detection dataset

**指標値比較**

Fast R-CNN mAP:35.9/
Faster R-CNN mAP:42.7

先行研究に比べ高いmAPを得られ、計算速度も早い

## (8) （アドバンス課題）Faster R-CNNよりも新しい物体検出の論文では、Faster R-CNNがどう引用されているか。

[M2Det](https://arxiv.org/pdf/1811.04533.pdf)

[CenterNet(Objects as Points)](https://arxiv.org/pdf/1904.07850.pdf)